In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
import tensorflow as tf
import numpy as np

# Check available GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Configure GPU settings
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Allow memory growth
    except RuntimeError as e:
        print(e)

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Hyperparameters
input_size = 1    # Only using the "Close" price for prediction
sequence_length = 60  # Using 60 previous days to predict the next days
target_sequences = [5, 15, 30]   # Predicting the next days
batch_size = 64
num_epochs = 50


# Downloading stock data from Yahoo Finance
# output: 2D array with a single column, (num, 1)
def get_stock_data(ticker):
    df = yf.download(ticker, start="2010-01-01", end="2023-01-01")
    return df['Close'].values.reshape(-1, 1)

# Preprocessing the data
def preprocess_data(data, sequence_length, target_sequences):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data)

    X, y = [], []

    set_zero_second = False
    set_zero_third = False
    # set the targets until the smallest target sequnece hits the end
    for i in range(len(data) - sequence_length - target_sequences[0] + 1):
        if not set_zero_second:
            if i + sequence_length + target_sequences[1] > len(data):
                set_zero_second = True
        if not set_zero_third:
            if i + sequence_length + target_sequences[2] > len(data):
                set_zero_third = True
        X.append(data[i:i+sequence_length])
        scales = []
        for scale, output_size in enumerate(target_sequences):
            if scale == 1 and set_zero_second:
                scales.append(np.zeros((output_size, 1)))
            elif scale == 2 and set_zero_third:
                scales.append(np.zeros((output_size, 1)))
            else:
                scales.append(data[i+sequence_length:i+sequence_length+output_size])
        scales_flat = [np.ravel(target) for target in scales]
        y.append(scales_flat)

    X = np.array(X)
    y = np.array(y, dtype=object)

    X = X.reshape((X.shape[0], X.shape[1], 1))
    y = np.array([np.concatenate(s) for s in y])
    return X, y, scaler

def split_data(X, y, train_size):
    split = int(0.8 * len(X))
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:], y[train_size:]

    return X_train, y_train, X_test, y_test



In [ ]:
data = get_stock_data('IBM')
X, y, scaler = preprocess_data(data, sequence_length, target_sequences)
X_train, y_train, X_test, y_test = split_data(X, y, int(0.8 * len(X)))

In [ ]:
# Architecture configuration

from keras.layers import LSTM, Dropout, Dense
import tensorflow as tf
from tensorflow.keras import Model
from keras.models import Sequential

"""
() = LSTM layer (features, return_sequences, skip_connection)
S = prediction layer
C = concatenation
"""
config = [
    (128, True, False),
    (128, True, True),
    (64, True, False),
    (64, True, True),
    (32, True, False),
    (32, True, False), # To this point is LSTMs (time_series interpretion)
    "S",
    (64, True, False),
    "C",
    (64, True, False),
    "S",
    (128, True, False),
    "C",
    (128, True, False),
    "S",
]

In [ ]:
class ScalePrediction(tf.keras.layers.Layer):
    def __init__(self, features, target_sequence):
        super(ScalePrediction, self).__init__()
        self.features = features
        self.target_sequence = target_sequence
        self.layers = []
        self.layers.append(LSTM(features))
        self.layers.append(ResidualBlock(features))
        self.layers.append(Dense(target_sequence))


    def call(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, features):
        super(ResidualBlock, self).__init__()
        self.layers = []
        self.layers.append(Dense(features, activation='relu'))
        self.layers.append(Dense(features, activation='relu'))


    def call(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x + inputs

# Output shape: (_, 50)
# 50: concatenated output sequence lengths e.g, 5 + 15 + 30
class customModel(Model):
    def __init__(self, sequence_length):
        super(customModel, self).__init__()
        self.sequence_length = sequence_length
        self.list_of_concats_A = []
        self.list_of_concats_B = []
        self.layers_list = self._create_layers()

    def call(self, inputs):
        outputs = []
        route_connections = []

        x = inputs
        for i, layer in enumerate(self.layers_list):
            if isinstance(layer, ScalePrediction):
                outputs.append(layer(x))
            else:
                x = layer(x)
                if i in self.list_of_concats_A:
                    route_connections.append(x)
                elif i in self.list_of_concats_B:
                    x = tf.concat([x, route_connections.pop()], axis=-1)
        # Concatenate all tensors along the last axis
        concatenated_outputs = tf.concat(outputs, axis=-1)
        flattened_outputs = tf.reshape(concatenated_outputs, [-1])
        return concatenated_outputs

    def _create_layers(self):
        layers = []
        count = 0
        for i, module in enumerate(config):
            if isinstance(module, tuple):
                if i == 0:
                    layers.append(LSTM(module[0], return_sequences=module[1], input_shape=(X_train.shape[1], X_train.shape[2])))
                else:
                    layers.append(LSTM(module[0], return_sequences=module[1]))
                if module[2]:
                    self.list_of_concats_A.append(i)
            elif module == "S":
                prev_features = config[i-1][0]
                layers.append(ScalePrediction(prev_features, target_sequences[count]))
                count += 1

            elif module == "C":
                self.list_of_concats_B.append(i-1)

        return layers

In [ ]:
# def custom_loss(y_true, y_pred):
#     loss = 0.0
#     batch_size = tf.shape(y_true)[0]
#     # Check shapes
#     tf.print("y_true shape:", tf.shape(y_true))
#     tf.print("y_pred shape:", tf.shape(y_pred))
#     for i in range(batch_size):
#         # get the predictions of each batch
#         y_pred_column = y_pred[i, :]
#         y_true_column = y_true[i, :]

#         # Only calculate loss for non-zero predictions
#         non_zero_mask = tf.not_equal(y_true_column, 0)  # Mask for non-zero values
#         y_pred_non_zero = tf.boolean_mask(y_pred_column, non_zero_mask)
#         y_true_non_zero = tf.boolean_mask(y_true_column, non_zero_mask)

#         # Calculate MSE for non-zero values
#         loss += tf.reduce_mean(tf.square(y_pred_non_zero - y_true_non_zero))
#     return loss


def custom_loss(y_true, y_pred):
    # Define a function to compute the loss for each batch
    def batch_loss_fn(batch_idx):
        # Get the predictions and true values of each batch
        y_pred_column = y_pred[batch_idx, :]
        y_true_column = y_true[batch_idx, :]

        # Only calculate loss for non-zero predictions
        non_zero_mask = tf.not_equal(y_true_column, 0)  # Mask for non-zero values
        y_pred_non_zero = tf.boolean_mask(y_pred_column, non_zero_mask)
        y_true_non_zero = tf.boolean_mask(y_true_column, non_zero_mask)

        # Calculate MSE for non-zero values
        return tf.reduce_mean(tf.square(y_pred_non_zero - y_true_non_zero))

    # Get the number of batches (first dimension)
    num_batches = tf.shape(y_true)[0]

    # Use tf.map_fn to apply the batch_loss_fn to each batch with fn_output_signature
    losses = tf.map_fn(batch_loss_fn, tf.range(num_batches), fn_output_signature=tf.float32)
    # Return the sum of losses
    return tf.reduce_sum(losses)


In [ ]:
model = customModel(sequence_length)
# (batch_size=None, timesteps=50, features=20)
input_shape = (1, 60, 1)
dummy_input = tf.random.normal(input_shape)  # Create a dummy input tensor
output = model(dummy_input)  # Call the model to build it
print(output)
# Print the model summary to see the total number of parameters
model.summary()

In [ ]:
# Model with multiple outputs
model.compile(optimizer='adam', loss=custom_loss)
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
# Train the model

model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=1)


In [ ]:
import matplotlib.pyplot as plt

indices = tf.range(0, len(X_test), 29)  # This will give you [  0  29  58  87 ..., 638]

# Make predictions
predicted_lstm = model.predict(X_test)
# Select the rows at these indices
selected_rows = predicted_lstm[indices]
prediction_reduced = selected_rows[:, -30:]
# Reshape the predicted and actual values to 2D (samples * time_steps, features)
predicted_lstm = prediction_reduced.reshape(-1, 1)

selected_rows = y_test[indices]
y_test_reduced = selected_rows[:, -30:]
y_test_actual = y_test_reduced.reshape(-1, 1)

# Inverse transform the predicted and actual values
predicted_lstm = scaler.inverse_transform(predicted_lstm)
y_test_actual = scaler.inverse_transform(y_test_actual)

# Plotting
plt.figure(figsize=(15, 6))
plt.plot(y_test_actual, label='Actual')
plt.plot(predicted_lstm, label='Predicted')
plt.title('Actual vs Predicted')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()